In [1]:
import pydoocs
from time import sleep, time
from datetime import datetime
import h5py
import numpy as np


In [2]:
class DoocsData:
    
    def __init__(self, doocs_addr):
        self._doocs_addr = doocs_addr
        self.data = []
        self.timestamps = []
        self.trainIDs = []
    
    def read(self):
        doocs_dict = pydoocs.read(self._doocs_addr)
        current_trainID = doocs_dict["macropulse"]
        sleep(0.001)
        if not self.trainIDs or current_trainID != self.trainIDs[-1]:
            self.data.append(doocs_dict["data"])
            self.timestamps.append(doocs_dict["timestamp"])
            self.trainIDs.append(current_trainID)


class DoocsRecorder:
    
    def __init__(self, filename, doocs_addrs, number_of_trains=36000):
        self._filename = filename
        self._doocs_addrs = doocs_addrs
        self._number_of_trains = number_of_trains
        self._recorded_trains = 0
        
    def record(self):
        chunck_sizes = self._chunck_trains(6000)
        for chunck in chunck_sizes:
            self._record_single_file(chunck)
        
    def _record_single_file(self, trains_per_file):
        self._create_doocs_data()
        try:
            first_dset = self._doocs_data[list(self._doocs_addrs.keys())[0]]
            while len(first_dset.trainIDs) < trains_per_file:
                self._update_doocs_data()
            self._write_to_h5()
            print(f"Done with recording {trains_per_file} trains.")
        except KeyboardInterrupt:
            self._write_to_h5()
            print("Stop requested by user: save recorded data.")
        
    def _create_doocs_data(self):
        self._doocs_data = {}
        for name in self._doocs_addrs:
            self._doocs_data[name] = DoocsData(self._doocs_addrs[name])
            
    def _update_doocs_data(self):
        for name, data in self._doocs_data.items():
                data.read()    
        
    def _write_to_h5(self):
        current_timestamp = self._current_timestamp()
        with h5py.File(f"{self._filename}_{current_timestamp}.h5", "w") as f:
            for name, data in self._doocs_data.items():            
                f.create_dataset(name+"/data", data=np.asarray(data.data))
                f.create_dataset(name+"/timestamp", data=np.asarray(data.timestamps))
                f.create_dataset(name+"/trainId", data=np.asarray(data.trainIDs))
    
    def _current_timestamp(self):
        date_time = datetime.fromtimestamp(time())
        return date_time.strftime("%Y%m%dT%H%M%S")
    
    def _chunck_trains(self, max_trains_per_file):
        chunck_sizes = self._number_of_trains//max_trains_per_file * [max_trains_per_file]     
        chunck_sizes += [self._number_of_trains % max_trains_per_file]
        return chunck_sizes



In [3]:
doocs_addr = {"A": "FLASH.DIAG/TOROID/7FL2DUMP/CHARGE.FLASH2",
              "B": "FLASH.DIAG/TIMER/FLASHCPUTIME1.0/NUMBER_BUNCHES.1"}
#               "C": "FLASH.FEL/ADC.ADQ.BL1/EXP1.CH00/CH00.DAQ.TD"
## bad for big data like traces & images -> skips trains in all channels

filename = "fsla"
number_of_trains = 101

recoder = DoocsRecorder(filename, doocs_addr, number_of_trains)
recoder.record()

Done with recording 101 trains.
